In [1]:
# import os
# os.environ['SNORKELDB'] = 'postgres://localhost:5432/babble_test_spouse'

# from snorkel import SnorkelSession
# session = SnorkelSession()

In [2]:
# import os

# from snorkel.models import candidate_subclass
# from snorkel.contrib.babble import ExplanationIO
# from snorkel.contrib.babble.models import configuration
# from tutorials.babble.spouse import SpouseModel

# config = configuration
# config['domain'] = 'spouse'
# config['splits'] = [0,1,2]
# config['babbler_split'] = 0
# config['max_docs'] = None
# config['parallelism'] = 1
# config['traditional'] = False
# config['majority_vote'] = True
# config['verbose'] = True
# config['display_marginals'] = True
# config['display_accuracies'] = True
# config['display_learned_accuracies'] = True

# Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

# sm = SpouseModel(session, Spouse, config)

In [3]:
# %time sm.parse(config=config)

In [4]:
# %time sm.extract(config=config)

In [5]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
# sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/babble/data/')
os.environ['SNORKELDB'] = 'postgres://localhost:5432/babble_test_spouse'
# os.environ['SNORKELDB'] = 'postgres://localhost:5432/babble_test_bike'

from snorkel import SnorkelSession
session = SnorkelSession()

In [6]:
from snorkel.models import candidate_subclass
from snorkel.contrib.babble import ExplanationIO
from mturk_processing import MTurkHelper

import os

# SPOUSE
Spouse = candidate_subclass('Spouse', ['person1', 'person2'])
candidate_class = Spouse
mode = 'text'
# fpath = os.environ['SNORKELHOME'] + '/tutorials/babble/spouse/data/grad_explanations.tsv'
fpath = os.environ['SNORKELHOME'] + '/tutorials/babble/spouse/data/mturk_explanations_all.tsv'
expio = ExplanationIO()
explanations = expio.read(fpath)

# BIKER
# Biker = candidate_subclass('Biker', ['person', 'bike'])
# candidate_class = Biker
# mode = 'image'
# helper = MTurkHelper(candidates, num_hits=40, workers_per_hit=3)
# output_csv_path= os.environ['SNORKELHOME'] + '/tutorials/babble/bike/data/VisualGenome_all_out.csv'
# explanations = helper.postprocess_visual(output_csv_path, verbose=False)

Read 445 explanations from /Users/bradenjh/repo...ata/mturk_explanations_all.tsv


In [7]:
candidates = session.query(candidate_class).filter(candidate_class.split == 0).all()
print(len(candidates))

23490


In [8]:
from snorkel.contrib.babble import link_explanation_candidates

explanations = link_explanation_candidates(explanations, candidates)

Building list of target candidate ids...
Collected 155 unique target candidate ids from 445 explanations.
Gathering desired candidates...
Found 155/155 desired candidates
Linking explanations to candidates...
Linked 445/445 explanations


In [9]:
# for exp in explanations:
#     if exp.condition == 'widow is mentioned between the two names':
#         explanations = [exp]
#         break
# print(explanations[0])

In [10]:
exp_iterator = iter(explanations)

In [11]:
exp = exp_iterator.next()
from snorkel.viewer import SentenceNgramViewer
print(exp.condition)
sv = SentenceNgramViewer([exp.candidate], session, n_per_page=3, height=150)
sv

Politicians


<IPython.core.display.Javascript object>

In [12]:
# exp.condition = 'since his widow is mentioned between the two names'

In [13]:
# explanations = [exp]
# explanations

In [14]:
from snorkel.contrib.babble import Babbler
user_lists = {}
babbler = Babbler(mode=mode, 
                  candidate_class=candidate_class, 
                  explanations=explanations,
                  string_format='implicit')

Created grammar with 473 rules


In [15]:
# for exp in explanations:
#     LF_dict = babbler.semparser.parse_and_evaluate(exp, show_erroring=True)
#     num_correct = len(LF_dict['correct'])
#     num_passing = len(LF_dict['passing'])
#     num_acceptable = num_correct + num_passing
#     if not num_acceptable > 0:
#         print(LF_dict)
#         babbler.semparser.grammar.print_chart()
#         parses = babbler.semparser.parse(exp, return_parses=True)
#         import pdb; pdb.set_trace()
#     else:
#         print('Acceptable: {}'.format(num_acceptable))

In [16]:
# for exp in explanations[:5]: print(exp.condition)

In [17]:
# from snorkel.contrib.babble import Explanation
# exp = Explanation("person1 equals person2", True, name='test_exp')
# homemades = [exp]

In [18]:
lfs = babbler.generate_lfs()

350 parses created from 260 out of 445 explanation(s)
Parsed 350 LFs from 445 explanations.


In [19]:
babbler.filter_duplicate_semantics()

Filtered to 260 LFs with duplicate semantics filter (90 filtered).


In [20]:
babbler.filter_consistency()

Filtered to 101 LFs with consistency filter (159 filtered).


In [21]:
babbler.get_parses(translate=True)

["call(in text(sentence()), 'couple')",
 "return 1 if call(contains('tied the knot'), text(sentence())) else 0",
 "return 1 if call(contains('husband'), text(sentence())) else 0",
 "call(in text(sentence()), 'husband')",
 "all(map(in text(sentence()), [text(arg1),'alerted his wife',text(arg2)]))",
 "call(in text(sentence()), 'his wife')",
 "all(map(in text(sentence()), [text(arg2),'his wife']))",
 "all(map(in text(sentence()), [text(arg1),'husband',text(arg2)]))",
 "all(map(in text(sentence()), [text(arg1),'tied the knot']))",
 "all(map(in text(sentence()), [text(arg1),text(arg2),'tied the knot']))",
 "call(in text(sentence()), 'ordered')",
 "call(in text(sentence()), 'prison guard')",
 "all(map(in text(sentence()), [text(arg1),'wife actress',text(arg2)]))",
 "return -1 if call(in text(between([arg1,arg2])), 'brother') else 0",
 'return 1 if call(contains(text(arg2)), text(sentence())) else 0',
 'return 1 if call(contains(text(arg1)), text(sentence())) else 0',
 "all(map(in text(senten

In [22]:
babbler.get_explanations()

[Explanation("Explanation107: False, there is a couple listed but these two are not it."),
 Explanation("Explanation12: True, It says they "tied the knot""),
 Explanation("Explanation123: True, It refers to Briath as Jodi's "husband""),
 Explanation("Explanation124: True, the highlighted names have husband before them"),
 Explanation("Explanation125: True, Person 1 alerted his wife Person 2 within the same sentence."),
 Explanation("Explanation126: True, Refers to "His wife""),
 Explanation("Explanation127: True, Refers to Jenny as "his wife""),
 Explanation("Explanation129: True, Patrick was the husband of Vernicia Ruguru"),
 Explanation("Explanation13: True, text states that person1 and person2 tied the knot"),
 Explanation("Explanation13: True, text states that person1 and person2 tied the knot"),
 Explanation("Explanation143: False, "ordered""),
 Explanation("Explanation145: False, Person is marrying the "prison guard""),
 Explanation("Explanation15: True, Depp's wife actress Amber

In [23]:
%time babbler.generate_label_matrix(split=1, parallelism=1)

Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 1min 9s, sys: 4.78 s, total: 1min 14s
Wall time: 2min 10s


<2456x101 sparse matrix of type '<type 'numpy.int64'>'
	with 186647 stored elements in Compressed Sparse Row format>

In [24]:
babbler.filter_uniform_signatures()

Filtered to 54 LFs with uniform signatures filter (47 filtered).


In [25]:
babbler.filter_duplicate_signatures()

Filtered to 21 LFs with duplicate signatures filter (33 filtered).


In [ ]:
from snorkel.annotations import load_gold_labels

L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_dev

In [ ]:
babbler.label_matrix.lf_stats(session, labels=L_gold_dev)